# OrgSci

In [499]:
class Section:
    def __init__(self, content, size):
        self.children = []
        self.size = size
        self.parent = None
        self.content = content
        
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.content.strip() == other.content.strip()
        return False
    
    def __repr__(self):
        return self.content
    
    def add_child(self, new_child):
        self.children.append(new_child)
        
    def has_child(self):
        return len(self.children) != 0
    
    def set_parent(self, new_parent):
        self.parent = new_parent
    
    def extend(self, content):
        self.content += f" {content}"
    
    def backtrack_add(self, content, size):
        curr = self
        
        while curr.size <= size:
            curr = curr.parent
        
        parent = curr
        cs = Section(content, size)
        parent.add_child(cs)
        cs.set_parent(parent)
                
        return cs
    def print_contents(self):
        if len(self.children) == 0:
            return self.content
        
        return self.content + "\n" + " \n\n ".join([child.print_contents() for child in self.children])
    
import pprint
import fitz
from fitz import Rect
import re
import pandas as pd



def get_sections(doc):

    main_section = Section("", 100)
    main_section   

    prev_size = 100
    curr_section = main_section
    nest = {}

    for page in doc:
        rect = Rect(page.rect.x0 + 40, page.rect.y0 + 60, page.rect.x1 - 40, page.rect.y1 - 40) 
        
        dict = page.get_text("dict", clip = rect)

        blocks = dict["blocks"]
        for block in blocks:
            if "lines" in block.keys():
                spans = block['lines']
                for span in spans:
                    data = span['spans']
                    for lines in data:
                        cur_size = round(lines['size'], 1)

                        # Manual Override for References
                        if lines['text'].strip() in ("Acknowledgements", "References", "Appendix", "Endnotes"):
                            cur = Section(lines['text'], cur_size)
                            curr_section = main_section.children[-1].children[-1]
                            curr_section.add_child(cur)
                            cur.set_parent(curr_section)
                            curr_section = cur

                            prev_size = round(lines['size'], 1)

                        elif cur_size > prev_size:
                            curr_section = curr_section.backtrack_add(lines['text'], cur_size)
                            prev_size = curr_section.size

                        elif cur_size == prev_size:
                            curr_section.extend(lines['text'])

                        else:  
                            cur = Section(lines['text'], cur_size)   
                            curr_section.add_child(cur)
                            cur.set_parent(curr_section)
                            curr_section = cur
                            prev_size = round(lines['size'], 1)

    # orgsci                            
    return main_section.children[-1].children[-1].children

def make_sections_dataframe(path):
    doc = fitz.open(path) # open a document
    
    # Get sections
    sections = get_sections(doc)
    sections = preprocess_sections(sections)
    
    content_nest = {}

    for section in sections:
        content_nest[section.content] = [section.print_contents()]

    sections_df = pd.DataFrame(content_nest, index = ["text"]).T
    sections_df.name = doc.name
    return sections, sections_df

def make_references_dataframe(sections, sections_df):
    references_dictionary = {}
    
    references_clean = text_preprocess_for_reference_matching(sections[-1].print_contents())

    for location, text in zip(sections_df.index[:-1], sections_df.values[:-1]):
        in_text_citations = get_in_text_citations(text.item())
        cleaned_in_text_citations = [
            item 
            for group in 
                [  
                    [c.strip()                             # Remove whitespace
                     for c in citation[1:-1].split(',')    # Remove '(' and ')'
                     if any(char.isdigit() for char in c)] # Remove any that doesn't have digits (year)

                    for citation in in_text_citations
                ]
            for item in group 
        ]
        author_year_pairs = list(filter(lambda x: x != None, map(process_citations, cleaned_in_text_citations)))
        references_dictionary = find_citation_matches(author_year_pairs, references_clean, references_dictionary, location)

    references_df = pd.DataFrame({k:[",".join(v)] for k,v in references_dictionary.items()}, index = ["section"]).T.reset_index(names = "reference")
    
    return references_df

def text_preprocess_for_reference_matching(references_text):
    # START searching ONCE References tag found
    references_dirty = re.sub("\n", " ", references_text)
    references_dirty = " ".join(references_dirty.split())
    references = re.sub("([0-9]|html|\))\s?\.", r"\g<0>\n", references_dirty)

    # Make list of references
    pattern = r"[A-ZÆØÅæøå][ÆØÅæøåA-Za-z]+.*[A-Z]{1,3},? .*\(\d{4}\).*[html|\d|\)]\."
    references_clean = re.findall(pattern, references)
    return references_clean
    

def get_in_text_citations(text):
    IN_TEXT_CITATION_REGEX = r"\([\w\s.,]+\s\d{3,4}\s?\)"
    return re.findall(IN_TEXT_CITATION_REGEX, text)

def process_citations(citation:str) -> (list,str):
    # case 1: 2 authors
    if " and " in citation:
        tokens = citation.split()
        year = tokens[-1]
        names = " ".join(tokens[:-1])
        names_split = names.split(" and ")
        return ((names_split[0].strip(), names_split[1].strip()), year.strip())

    # case 2: et al
    if "et al." in citation:
        tokens = citation.split("et al.")
        return ([tokens[0].strip()], tokens[1].strip())
    
    # case 3: 1 author
    else:
        split = citation.split()
        if len(split) == 1:
            return None
        else:
            tokens = citation.split()
            year = tokens[-1]
            author = " ".join(tokens[:-1])
            return ([author.strip()], year.strip())
        
        
def find_citation_matches(author_year_pairs, full_references, data, location):
    for author_year_pair in author_year_pairs:
        authors, year = author_year_pair
        for reference in full_references:
            match = True
            if f"({year})" in reference:
                for author in authors:
                    if author not in reference:
                        match = False
                if match:
                    dict_value = data.get(reference, [])
                    if dict_value == []:
                        data[reference] = []
                    if location not in dict_value:
                        data[reference] = data.get(reference, []) + [location]
            else:
                continue
    return data

def convert_pdf_to_dataframes(path) -> (pd.DataFrame, pd.DataFrame):
    """Returns (sections_df, references_df)"""
    sections, sections_df = make_sections_dataframe(path)
    references_df = make_references_dataframe(sections, sections_df)
    return sections_df, references_df

def preprocess_sections(sections):
    add_new_section = False
    
    # Preprocess sections
    abstract_section = Section("Abstract. ", 0)
    abstract_index = sections.index(abstract_section)
    abstract_text = sections.pop(abstract_index + 1)
    sections[abstract_index].add_child(abstract_text)
    
    sections_tmp = sections.copy()
    earliest_index = 1000
    txt = ""
    
    # Fit all text that belongs in paragraph into one "Introduction" paragraph
    for idx, section in enumerate(sections_tmp):
        if len(section.content) > 100:
            add_new_section = True
            if min(earliest_index, idx) != earliest_index:
                earliest_index = idx
            sections.remove(section)
            txt += section.content
            
    
    if add_new_section:
        new_section = Section("Introduction", 20)
        new_section.add_child(Section(txt, 15))
        sections[earliest_index] = new_section

    sections = sections[abstract_index:]
    
    return sections

import fitz
import os

ORGSCI_PATH = "data/orgsci"
orgsci_pdfs = [f for f in os.listdir(ORGSCI_PATH) if f.endswith('pdf')]
path = os.path.join(ORGSCI_PATH, orgsci_pdfs[0])

sections_df, references_df = convert_pdf_to_dataframes(path)
sections_df

,text
Abstract.,Abstract. \nIn the wake of media hype about ar...
Introduction,Introduction\nThe most promising uses of AI wi...
Technology as a Tool: How Technologies Support Tasks and Work,Technology as a Tool: How Technologies Suppor...
Technology as a Medium: How Technologies Enable Collaboration Between Groups,Technology as a Medium: How Technologies Enab...
Technology as a Counterpart: How People and Technologies Interact Within a System of Work,Technology as a Counterpart: How People and T...
Taking a Counterpart Perspective: Methodological Challenges,Taking a Counterpart Perspective: Methodologi...
A Relational Approach to Ethnography and Studying AI,A Relational Approach to Ethnography and Stud...
Conclusion,Conclusion\nAlthough positive narratives about...
Endnotes,Endnotes\n1 \n\n Machine learning algorithms ...
References,"References\nAjunwa I, Greene D (2019) Platform..."


# Annurev

In [200]:
class Section:
    def __init__(self, content, size):
        self.children = []
        self.size = size
        self.parent = None
        self.content = content
        
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.content.strip() == other.content.strip()
        return False
    
    def __repr__(self):
        return self.content
    
    def add_child(self, new_child):
        self.children.append(new_child)
        
    def has_child(self):
        return len(self.children) != 0
    
    def set_parent(self, new_parent):
        self.parent = new_parent
    
    def extend(self, content):
        self.content += f" {content}"
    
    def backtrack_add(self, content, size):
        curr = self
        
        while curr.size <= size:
            curr = curr.parent
        
        parent = curr
        cs = Section(content, size)
        parent.add_child(cs)
        cs.set_parent(parent)
                
        return cs
    def print_contents(self):
        if len(self.children) == 0:
            return self.content
        
        return self.content + "\n" + " \n\n ".join([child.print_contents() for child in self.children])
    
import pprint
import fitz
from fitz import Rect
import re
import pandas as pd



def get_sections(doc):

    main_section = Section("", 100)
    main_section   

    prev_size = 100
    curr_section = main_section
    nest = {}

    for page in doc:
        if page.number not in (doc[-1].number, doc[-2].number):
            if page.number == 0:
                rect = Rect(page.rect.x0 + 20, page.rect.y0 + 250, page.rect.x1 - 20, page.rect.y1 - 20) 
            else:
                rect = Rect(page.rect.x0 + 20, page.rect.y0 + 20, page.rect.x1 - 20, page.rect.y1 - 20) 
            
            dict = page.get_text("dict", clip = rect)

            blocks = dict["blocks"]
            for block in blocks:
                if "lines" in block.keys():
                    spans = block['lines']
                    for span in spans:
                        data = span['spans']
                        for lines in data:
                            cur_size = round(lines['size'], 1)

                            # Manual Override for References
                            if lines['text'].strip() in ("Acknowledgements", "References", "Appendix", "Endnotes"):
                                cur = Section(lines['text'], cur_size)
                                curr_section = main_section.children[-1].children[-1]
                                curr_section.add_child(cur)
                                cur.set_parent(curr_section)
                                curr_section = cur

                                prev_size = round(lines['size'], 1)

                            elif cur_size > prev_size:
                                curr_section = curr_section.backtrack_add(lines['text'], cur_size)
                                prev_size = curr_section.size

                            elif cur_size == prev_size:
                                curr_section.extend(lines['text'])

                            else:  
                                cur = Section(lines['text'], cur_size)   
                                curr_section.add_child(cur)
                                cur.set_parent(curr_section)
                                curr_section = cur
                                prev_size = round(lines['size'], 1)

    # orgsci                            
    return main_section

def preprocess_sections(sections):
    # Preprocess sections
    abstract_section = Section("Abstract", 0)

    abstract_index = sections.index(abstract_section)
    abstract_text = sections.pop(abstract_index + 1)
    sections[abstract_index].add_child(abstract_text)

    sections = sections[abstract_index:]
    
    return sections

def make_sections_dataframe(path):
    doc = fitz.open(path) # open a document
    
    # Get sections
    sections = get_sections(doc)
    sections = preprocess_sections(sections)
    
    content_nest = {}

    for section in sections:
        content_nest[section.content] = [section.print_contents()]

    sections_df = pd.DataFrame(content_nest, index = ["text"]).T
    sections_df.name = doc.name
    return sections, sections_df

def make_references_dataframe(sections, sections_df):
    references_dictionary = {}
    
    references_clean = text_preprocess_for_reference_matching(sections[-1].print_contents())

    for location, text in zip(sections_df.index[:-1], sections_df.values[:-1]):
        in_text_citations = get_in_text_citations(text.item())
        cleaned_in_text_citations = [
            item 
            for group in 
                [  
                    [c.strip()                             # Remove whitespace
                     for c in citation[1:-1].split(',')    # Remove '(' and ')'
                     if any(char.isdigit() for char in c)] # Remove any that doesn't have digits (year)

                    for citation in in_text_citations
                ]
            for item in group 
        ]
        author_year_pairs = list(filter(lambda x: x != None, map(process_citations, cleaned_in_text_citations)))
        references_dictionary = find_citation_matches(author_year_pairs, references_clean, references_dictionary, location)

    references_df = pd.DataFrame({k:[",".join(v)] for k,v in references_dictionary.items()}, index = ["section"]).T.reset_index(names = "reference")
    
    return references_df

def text_preprocess_for_reference_matching(references_text):
    # START searching ONCE References tag found
    references_dirty = re.sub("\n", " ", references_text)
    references_dirty = " ".join(references_dirty.split())
    references = re.sub("([0-9]|html|\))\s?\.", r"\g<0>\n", references_dirty)

    # Make list of references
    pattern = r"[A-ZÆØÅæøå][ÆØÅæøåA-Za-z]+.*[A-Z]{1,3},? .*\(\d{4}\).*[html|\d|\)]\."
    references_clean = re.findall(pattern, references)
    return references_clean
    

def get_in_text_citations(text):
    IN_TEXT_CITATION_REGEX = r"\([\w\s.,]+\s\d{3,4}\s?\)"
    return re.findall(IN_TEXT_CITATION_REGEX, text)

def process_citations(citation:str) -> (list,str):
    # case 1: 2 authors
    if " and " in citation:
        tokens = citation.split()
        year = tokens[-1]
        names = " ".join(tokens[:-1])
        names_split = names.split(" and ")
        return ((names_split[0].strip(), names_split[1].strip()), year.strip())

    # case 2: et al
    if "et al." in citation:
        tokens = citation.split("et al.")
        return ([tokens[0].strip()], tokens[1].strip())
    
    # case 3: 1 author
    else:
        split = citation.split()
        if len(split) == 1:
            return None
        else:
            tokens = citation.split()
            year = tokens[-1]
            author = " ".join(tokens[:-1])
            return ([author.strip()], year.strip())
        
        
def find_citation_matches(author_year_pairs, full_references, data, location):
    for author_year_pair in author_year_pairs:
        authors, year = author_year_pair
        for reference in full_references:
            match = True
            if f"({year})" in reference:
                for author in authors:
                    if author not in reference:
                        match = False
                if match:
                    dict_value = data.get(reference, [])
                    if dict_value == []:
                        data[reference] = []
                    if location not in dict_value:
                        data[reference] = data.get(reference, []) + [location]
            else:
                continue
    return data

def convert_pdf_to_dataframes(path) -> (pd.DataFrame, pd.DataFrame):
    """Returns (sections_df, references_df)"""
    sections, sections_df = make_sections_dataframe(path)
    references_df = make_references_dataframe(sections, sections_df)
    return sections_df, references_df 

In [425]:
def get_sections(doc):

    main_section = Section("", 100)

    prev_size = 100
    curr_section = main_section
    nest = {}

    for page in doc:
        if page.number not in (doc[-1].number, doc[-2].number):
            rect = Rect(page.rect.x0 + 20, page.rect.y0 + 20, page.rect.x1 - 20, page.rect.y1 - 30)
            
            dict = page.get_text("dict", clip = rect)
            blocks = dict["blocks"]
            for block in blocks:
                if "lines" in block.keys():
                    spans = block['lines']
                    for span in spans:
                        data = span['spans']
                        for lines in data:
                            cur_size = round(lines['size'], 2)

                            # Manual Override for References
                            if lines['text'].strip() in ("Abstract", "Keywords", "LITERATURE CITED"):
                                cur = Section(lines['text'], cur_size)
                                curr_section = main_section.children[-1].children[-1]
                                curr_section.add_child(cur)
                                cur.set_parent(curr_section)
                                curr_section = cur

                                prev_size = round(lines['size'], 2)

                            elif cur_size > prev_size:
                                curr_section = curr_section.backtrack_add(lines['text'], cur_size)
                                prev_size = curr_section.size

                            elif cur_size == prev_size:
                                curr_section.extend(lines['text'])

                            else:  
                                cur = Section(lines['text'], cur_size)   
                                curr_section.add_child(cur)
                                cur.set_parent(curr_section)
                                curr_section = cur
                                prev_size = round(lines['size'], 2)

    # orgsci                            
    return main_section.children[-1].children

In [430]:
sections

[John P. Hausknecht, Further]

In [461]:
annurev_path = "data/annurev-orgpsych"
annurev_pdfs = [f for f in os.listdir(annurev_path) if f.endswith('pdf')]
doc = fitz.open(os.path.join(annurev_path, annurev_pdfs[1])) # open a document
print(doc)
sections = get_sections(doc)
# sections = preprocess_sections(sections)
sections[1].children
# content_nest = {}

# for section in sections:
#     content_nest[section.content] = [section.print_contents()]

# sections_df = pd.DataFrame(content_nest, index = ["text"]).T
# sections_df

Document('data/annurev-orgpsych/Hausknecht_2017_Annurev-Orgpsych_Collective Turnover.pdf')


[INTRODUCTION,
 DEFINITIONS AND CONSTRUCT ISSUES,
 THEORETICAL ADVANCEMENTS,
 CHOOSING A COLLECTIVE TURNOVER MEASURE,
 Context-Emergent Turnover Theory,
 Turnover Capacity,
 RECENT EMPIRICAL CONTRIBUTIONS,
 Meta-Analyses,
 Original Articles,
 FUTURE RESEARCH NEEDS,
 FIFTEEN PROMISING RESEARCH QUESTIONS FOR FUTURE COLLECTIVE TURNOVER RESEARCH,
 Expand the Study of Context (Research Questions 7, 8, 12, 13, and 15),
 Move Beyond the United States (Research Questions 2, 6, 11, 13, and 15),
 Examine Events and Interventions (Research Questions 4, 5, 9, and 11),
 Measure Process (Research Questions 3, 6, and 14),
 Expand, Validate, and Reﬁne Measures (Research Questions 1, 2, 5, and 10),
 CONCLUSION,
 DISCLOSURE STATEMENT,
 LITERATURE CITED]